# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal

Loaded lib <CDLL './ddsolver/libdds.so', handle 55c95737d3c0 at 0x7f0d7f834990>


In [3]:
models = Models.load('../models')   # loading neural networks

INFO:tensorflow:Restoring parameters from ../models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ../models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ../models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ../models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ../models/lefty_model_tiny/lefty-1000000
INFO:tensorflow:Restoring parameters from ../models/dummy_model_tiny/dummy-1000000
INFO:tensorflow:Restoring parameters from ../models/righty_model_tiny/righty-1000000
INFO:tensorflow:Restoring parameters from ../models/decl_model_tiny/decl-1000000


### Running through an example

In [4]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

# you sit West and hold:
hand = '73.KJ83.AT2.T962'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['PAD_START', '1D', '1S']

# what's your bid?

In [5]:
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)

bot_bid.bid(['PAD_START', '1D', '1S']).bid

'X'

In [6]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(['PAD_START', '1D', '1S', 'X', '2S', '3H', 'PASS'])
bid.bid

'4H'

In [7]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': '4H',
  'insta_score': 0.6589298248291016,
  'expected_score': 38.16571872706945},
 {'call': 'PASS',
  'insta_score': 0.3114378750324249,
  'expected_score': 15.413541678671342}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [8]:
bid.samples

['xxxx.Qxx.Q8xx.KQ Q8.A9xx.KJxx.Axx AKJT9.Tx.9x.J8xx xx.KJ8x.ATx.T9xx',
 'KJ9x.Txx.8xx.Qxx 8x.AQx.KJxxx.KJx AQTxx.9xx.Q9.A8x xx.KJ8x.ATx.T9xx',
 'T8x.QT.KJ9x.QJ8x Qxx.A9xx.Q8xx.Ax AKJ9x.xxx.xx.Kxx xx.KJ8x.ATx.T9xx',
 '8xx.9x.K8xx.KQ8x AT9.Axxx.Q9xx.Ax KQJxx.QTx.Jx.Jxx xx.KJ8x.ATx.T9xx',
 '8xx.A9xx.QJxx.xx ATx.QTxx.K9xx.AJ KQJ9x.x.8x.KQ8xx xx.KJ8x.ATx.T9xx',
 'KTxx.Q9x.xxx.Qxx Ax.ATxx.KQJ9x.8x QJ98x.xx.8x.AKJx xx.KJ8x.ATx.T9xx',
 'QJx.9xxx.xxx.K8x AT.ATxx.QJ8x.QJx K98xxx.Q.K9x.Axx xx.KJ8x.ATx.T9xx',
 'JTx.x.K98x.K8xxx K98.AQ9x.QJxxx.Q AQxxx.Txxx.x.AJx xx.KJ8x.ATx.T9xx',
 'Txx.9xx.QJx.Axxx KQ9.AQTx.8xxxx.Q AJ8xx.xx.K9.KJ8x xx.KJ8x.ATx.T9xx',
 'QT9.Qxx.98xx.AJx KJ8.AT9x.KJxxx.8 Axxxx.xx.Q.KQxxx xx.KJ8x.ATx.T9xx']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [9]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', '1D', '1S', 'X', '2S', '3H', 'PASS'])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

8xx.xx.xxx.AKJxx K9x.AQTx.KQJ98.8 AQJTx.9xx.xx.Qxx xx.KJ8x.ATx.T9xx
KJ8.xx.8xx.J8xxx Q9x.Axxx.KQJ9x.K ATxxx.QT9.xx.AQx xx.KJ8x.ATx.T9xx
xxx.9xx.9x.AQJxx A98.Axxx.KJ8xx.x KQJTx.QT.Qxx.K8x xx.KJ8x.ATx.T9xx
KTxx.9xx.9x.Qxxx 8x.ATxx.KJ8x.AJ8 AQJ9x.Qx.Qxxx.Kx xx.KJ8x.ATx.T9xx
Q9x.Qxx.Q9x.Kxxx ATx.ATxx.Kxxx.Jx KJ8xx.9x.J8x.AQ8 xx.KJ8x.ATx.T9xx
Kxx.Txx.KJ98.Jxx AT.A9xx.Qxxx.A8x QJ98xx.Qx.xx.KQx xx.KJ8x.ATx.T9xx
JT8x.T9x.Qx.A8xx 9x.AQxx.KJ8xxx.Q AKQxx.xx.9x.KJxx xx.KJ8x.ATx.T9xx
Kxx.T9xx.QJxx.xx ATx.AQxx.98xx.K8 QJ98x.x.Kx.AQJxx xx.KJ8x.ATx.T9xx
Jxx.ATx.Jxxx.Jxx AK.Qxxx.K9xx.Qxx QT98xx.9x.Q8.AK8 xx.KJ8x.ATx.T9xx
AJ9.9xx.Q8x.Q8xx 8x.QTxx.K9xx.AKJ KQTxxx.Ax.Jxx.xx xx.KJ8x.ATx.T9xx
Qxx.QT.J8xx.Qxxx A98.A9xx.KQxx.Jx KJTxx.xxx.9x.AK8 xx.KJ8x.ATx.T9xx
K9x.Qx.J98x.K8xx xx.Axxx.Qxxx.AQx AQJT8x.T9x.Kx.Jx xx.KJ8x.ATx.T9xx
Jxx.Qxx.QJx.QJxx KTx.AT9x.K9xx.K8 AQ98x.xx.8xx.Axx xx.KJ8x.ATx.T9xx
KTx.9xx.Kxxx.QJx AJ8.AQxx.Q9xx.8x Q9xxx.Tx.J8.AKxx xx.KJ8x.ATx.T9xx
QT9.9xx.Qx.KJ8xx AKx.Qxxx.J98xx.Q J8xxx.AT.Kxx.A

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [10]:
hands = random_deal().split()
hands # N, E, S, W

['A8.Q752.T54.JT63',
 'K54.T643.A92.A72',
 'JT932.K9.Q86.854',
 'Q76.AJ8.KJ73.KQ9']

In [11]:
bidder_bots = [BotBid([False, False], hand, models) for hand in hands]

In [12]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS',
 'PASS',
 'PASS',
 '1N',
 'PASS',
 '2C',
 'PASS',
 '2D',
 'PASS',
 '3N',
 'PASS',
 'PASS',
 'PASS']